In [28]:
%load_ext lab_black
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import re
import numpy as np
import collections

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [29]:
input = """NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C"""

with open("../inputs/d14.txt", "r") as input_file:
    test_input = input_file.read()

In [30]:
template, actions = input.split("\n\n")[0], input.split("\n\n")[1].splitlines()

In [31]:
def update_template(template, actions):
    char_map = {
        x: y
        for x, y in zip(
            [a.split(" -> ")[0] for a in actions], [a.split(" -> ")[1] for a in actions]
        )
    }
    str_res = ""
    for i in range(len(template) - 1):
        sbs = template[i : i + 2]
        if sbs in char_map:
            str_res += sbs[0] + char_map[sbs]
        else:
            str_res += sbs[0]

    str_res += template[-1]
    return str_res

In [32]:
def get_char_counter(input, steps, print_template=False):
    template, actions = input.split("\n\n")[0], input.split("\n\n")[1].splitlines()
    for i in range(steps):
        template = update_template(template, actions)
        if print_template:
            print(f"step: {i+1}: {template}")
    return collections.Counter(template)

In [33]:
def get_max_min_count_diff(input, steps, print_template=False):
    final_template = get_char_counter(input, steps, print_template)
    counts = [count for count in final_template.values()]
    return np.max(counts) - np.min(counts)

In [34]:
get_max_min_count_diff(input, 5, True)

step: 1: NCNBCHB
step: 2: NBCCNBBBCBHCB
step: 3: NBBBCNCCNBBNBNBBCHBHHBCHB
step: 4: NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB
step: 5: NBBNBBNBBBNBBNBBCNCCNBBBCCNBCNCCNBBNBBNBBNBBNBBNBNBBNBBNBBNBBNBBCHBHHBCHBHHNHCNCHBCHBNBBCHBHHBCHB


33

# Par 2

In [35]:
template, actions = input.split("\n\n")[0], input.split("\n\n")[1].splitlines()

In [36]:
template

'NNCB'

In [37]:
pairs_count = {}
for i in range(len(template) - 1):
    if template[i : i + 2] not in pairs_count:
        pairs_count[template[i : i + 2]] = 0
    pairs_count[template[i : i + 2]] += 1

pairs_count

{'NN': 1, 'NC': 1, 'CB': 1}

In [38]:
def update_pairs_count(action, pairs_count, local_pairs_count):
    pair, char = action.split(" -> ")[0], action.split(" -> ")[1]
    if pair not in pairs_count or pairs_count[pair] == 0:
        return
    pairs_to_add = [pair[0] + char, char + pair[1]]
    for pta in pairs_to_add:
        if pta not in local_pairs_count:
            local_pairs_count[pta] = 0
        local_pairs_count[pta] += pairs_count[pair]
    local_pairs_count[pair] -= pairs_count[pair]

In [39]:
local_pairs_count = pairs_count.copy()
for act in actions:
    update_pairs_count(act, pairs_count, local_pairs_count)

In [40]:
pairs_count
local_pairs_count

{'NN': 1, 'NC': 1, 'CB': 1}

{'NN': 0, 'NC': 1, 'CB': 0, 'CH': 1, 'HB': 1, 'CN': 1, 'NB': 1, 'BC': 1}